In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from keras import optimizers
from keras.applications import VGG16
from keras.applications import ResNet50
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import matplotlib.font_manager as fm
from keras.models import load_model

Using TensorFlow backend.


In [2]:
width = 224
height = 224

In [3]:
base_dir = '/Users/joannakang/Desktop/study/cat-and-dog'

In [4]:
test_dir = os.path.join(base_dir, 'test')
train_dir = os.path.join(base_dir, 'train')

In [5]:
class_name = os.listdir(test_dir)

In [6]:
class_name

['dogs', 'cats', '.DS_Store']

In [7]:
class_name.remove('.DS_Store')

In [8]:
train_image = []
train_label = []

In [9]:
test_image = []
test_label = []

In [10]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(class_name)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

print(onehot_encoded)

[[0. 1.]
 [1. 0.]]


/Users/joannakang/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [11]:
for i in range(len(class_name)):
    path = os.path.join(train_dir, class_name[i])
    img_list = os.listdir(path)
    for j in img_list:
        img = os.path.join(path, j)
        img = cv2.imread(img, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (width, height), interpolation = cv2.INTER_CUBIC)
        img = img.reshape((width, height, 3))
        train_image.append(img)
        train_label.append(onehot_encoded[i])

In [12]:
for i in range(len(class_name)):
    path = os.path.join(test_dir, class_name[i])
    img_list = os.listdir(path)
    for j in img_list: 
        img = os.path.join(path,j)
        img = cv2.imread(img, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (width, height), interpolation = cv2.INTER_CUBIC)
        img = img.reshape((width, height, 3))
        test_image.append(img)
        test_label.append(onehot_encoded[i])        

In [13]:
print(len(train_image))
print(len(train_label))
print(len(test_image))
print(len(test_label))

8005
8005
2023
2023


In [14]:
train_image, val_image, train_label, val_label = train_test_split(train_image, train_label, test_size=0.2, random_state=32)

In [15]:
print(len(train_image))
print(len(val_image))
print(len(test_image))

6404
1601
2023


In [16]:
train_image = np.array(train_image)
train_label = np.array(train_label)
val_image = np.array(val_image)
val_label = np.array(val_label)
test_image = np.array(test_image)
test_label = np.array(test_label)

In [17]:
print(train_image.shape)
print(test_image.shape)
print(val_image.shape)

(6404, 224, 224, 3)
(2023, 224, 224, 3)
(1601, 224, 224, 3)


In [29]:
pre_trained_model = VGG16(include_top = False, input_shape = (224, 224, 3))

In [30]:
pre_trained_model.trainable = True

In [31]:
set_trainable = False
for layer in pre_trained_model.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [32]:
pre_trained_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [33]:
# FCN for VGG16
model = Sequential()
model.add(pre_trained_model)
model.add(Flatten())

model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(train_label.shape[1], activation = 'sigmoid'))

In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                802848    
_________________________________________________________________
dense_5 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 34        
Total params: 15,518,098
Trainable params: 7,882,834
Non-trainable params: 7,635,264
_________________________________________________________________


In [35]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
history = model.fit(x = train_image, y = train_label, validation_data = (val_image, val_label), batch_size = 5, epochs = 1)

Train on 6404 samples, validate on 1601 samples
Epoch 1/1
2740/6404 [===========>..................] - ETA: 16:04 - loss: 7.9414 - accuracy: 0.5073

KeyboardInterrupt: 